In [0]:
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [0]:
text=open('/content/Extractive-based-summarization/rahul dravid.txt', 'r')

In [0]:
data=text.readlines()

In [0]:
text.close()

In [0]:
def preprocess(text):
    new=text.lower()
    new=re.sub("[\(\[].*?[\)\]]", "", new)
    new=re.sub("'s","",new)
    new=re.sub("[^'0-9.a-zA-Z]", " ", new)
    token=new.split()
    return (" ".join(token)).strip()

In [0]:
word=[]
for i in data:
    word.append(preprocess(i))
    

In [130]:
word

['dravid was born in a marathi deshastha brahmin family in indore madhya pradesh. his family later moved to bangalore karnataka where he was raised. his mother tongue is marathi. dravid father sharad dravid worked for a company that makes jams and preserves giving rise to the later nickname jammy. his mother pushpa was a professor of architecture at the university visvesvaraya college of engineering bangalore. dravid has a younger brother named vijay. he did his schooling at st. joseph boys high school bangalore and earned a degree in commerce from st. joseph college of commerce bangalore. he was selected to india national cricket team while studying mba in st joseph college of business administration. he is fluent in several languages marathi kannada english and hindi.',
 'dravid started playing cricket at the age of 12 and represented karnataka at the under 15 the under 17 and the under 19 levels. former cricketer keki tarapore first noticed dravid talent while coaching at a summer c

In [0]:
sent=[]
for i in word:
    sentence=i.split(".")
    for i in sentence:
        if(i!=''):
            sent.append(i.strip())
       
    

In [132]:
sent

['dravid was born in a marathi deshastha brahmin family in indore madhya pradesh',
 'his family later moved to bangalore karnataka where he was raised',
 'his mother tongue is marathi',
 'dravid father sharad dravid worked for a company that makes jams and preserves giving rise to the later nickname jammy',
 'his mother pushpa was a professor of architecture at the university visvesvaraya college of engineering bangalore',
 'dravid has a younger brother named vijay',
 'he did his schooling at st',
 'joseph boys high school bangalore and earned a degree in commerce from st',
 'joseph college of commerce bangalore',
 'he was selected to india national cricket team while studying mba in st joseph college of business administration',
 'he is fluent in several languages marathi kannada english and hindi',
 'dravid started playing cricket at the age of 12 and represented karnataka at the under 15 the under 17 and the under 19 levels',
 'former cricketer keki tarapore first noticed dravid tal

In [0]:
tokenize=[]
for i in sent:
    tokenize.append(i.split())

In [134]:
tokenize

[['dravid',
  'was',
  'born',
  'in',
  'a',
  'marathi',
  'deshastha',
  'brahmin',
  'family',
  'in',
  'indore',
  'madhya',
  'pradesh'],
 ['his',
  'family',
  'later',
  'moved',
  'to',
  'bangalore',
  'karnataka',
  'where',
  'he',
  'was',
  'raised'],
 ['his', 'mother', 'tongue', 'is', 'marathi'],
 ['dravid',
  'father',
  'sharad',
  'dravid',
  'worked',
  'for',
  'a',
  'company',
  'that',
  'makes',
  'jams',
  'and',
  'preserves',
  'giving',
  'rise',
  'to',
  'the',
  'later',
  'nickname',
  'jammy'],
 ['his',
  'mother',
  'pushpa',
  'was',
  'a',
  'professor',
  'of',
  'architecture',
  'at',
  'the',
  'university',
  'visvesvaraya',
  'college',
  'of',
  'engineering',
  'bangalore'],
 ['dravid', 'has', 'a', 'younger', 'brother', 'named', 'vijay'],
 ['he', 'did', 'his', 'schooling', 'at', 'st'],
 ['joseph',
  'boys',
  'high',
  'school',
  'bangalore',
  'and',
  'earned',
  'a',
  'degree',
  'in',
  'commerce',
  'from',
  'st'],
 ['joseph', 'colle

In [0]:
model_w2v = gensim.models.Word2Vec(
            tokenize,
            size=200, # desired no. of features/independent variables 
            window=5, # context window size
            min_count=2,
            sg = 0, # 1 for cbow model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
seed = 34)

In [136]:
model_w2v.train(tokenize, total_examples= len(tokenize), epochs=model_w2v.epochs)

(19002, 35240)

In [0]:
def wordw(token,size):
    vec=np.zeros(size).reshape((1,size))
    count=0
    for i in token:
        try:
            vec += model_w2v[i].reshape((1,size))
            count += 1
        except keyError:
              continue
    if count!=0:
        vec /= count
        
    return vec
    

In [142]:
word_array=np.zeros((len(tokenize),200))
for i in range(len(tokenize)):
    word_array[i,:]=wordw(tokenize[i],200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


NameError: ignored

In [0]:
sim_mat = np.zeros([len(word_array), len(word_array)])

In [0]:
for i in range(len(word_array)):
    for j in range(len(word_array)):
      if i != j:
          sim_mat[i][j] = cosine_similarity(word_array[i].reshape(1,200), word_array[j].reshape(1,200))[0,0]

In [0]:
nx_graph = nx.from_numpy_array(sim_mat)
#compute pagerank scores
scores = nx.pagerank(nx_graph)
#sort the scores
sorted_x = sorted(scores.items(), key=lambda kv: kv[1],reverse=True)


In [0]:
sent_list=[]
for i in sorted_x:
    sent_list.append(i[0])


In [160]:
num=10
summary=''
for i in range(num):
    summary=summary+sent[sent_list[i]]+'.'
print(summary.)

dravid was born in a marathi deshastha brahmin family in indore madhya pradesh.his family later moved to bangalore karnataka where he was raised.his mother tongue is marathi.dravid father sharad dravid worked for a company that makes jams and preserves giving rise to the later nickname jammy.his mother pushpa was a professor of architecture at the university visvesvaraya college of engineering bangalore.dravid has a younger brother named vijay.he did his schooling at st.joseph boys high school bangalore and earned a degree in commerce from st.joseph college of commerce bangalore.he was selected to india national cricket team while studying mba in st joseph college of business administration.
